In [1]:
import pandas as pd
import os, datetime, importlib 
calendar = pd.read_excel('')
source = ""
print(os.listdir(source))

['01-Aug-20', '01-Feb-21.txt', '01-Jun-21.txt', '01-Mar-21.txt', '02-Aug-21.txt', '03-May-21.txt', '04-Jan-21.txt', '05-Apr-21.txt', '06-Sep-21.txt', '07-Dec-20.txt', '07-Jun-21.txt', '08-Feb-21.txt', '08-Mar-21.txt', '10-Aug-21.txt', '10-Jan-22.txt', '10-May-21.txt', '11-Jan-21.txt', '12-Apr-21.txt', '12-Jul-21.txt', '12-Oct-21.txt', '13-Dec-21.txt', '13-Jun-21.txt', '13-Sep-21.txt', '14-Dec-20.txt', '15-Feb-21.txt', '15-Mar-21.txt', '16-Nov-21.txt', '17-Aug-21.txt', '17-May-21.txt', '17-Oct-20.txt', '18-Jan-21.txt', '19-Apr-21.txt', '19-Jul-21.txt', '19-Oct-21.txt', '20-Dec-21.txt', '20-Jun-21.txt', '20-Sep-21.txt', '21-Dec-20.txt', '22-Feb-21.txt', '22-Mar-21.txt', '23 Aug 2020.csv', '23-Aug-20', '23-Nov-20.txt', '23-Nov-21.txt', '24-Aug-21 - P45E.xlsx', '24-Aug-21.txt', '24-Oct-20.txt', '25-Jan-21.txt', '25-May-21.txt', '26-Jul-21.txt', '26-Oct-21.txt', '27-Dec-21.txt', '27-Jun-21.txt', '28-Dec-20.txt', '28-Sep-21.txt', '29-Mar-21.txt', '3-Jan-22.txt', '30-Aug-21.txt', '30-Nov-20.t

In [2]:
def get_csv(filename):
    print(f'Retrieving {filename}...')
    data = pd.read_csv(source+filename,sep=',',encoding='unicode_escape')
    data['Date/Time'] = pd.to_datetime(data['Date/Time'],format='%d/%m/%Y %H:%M:%S')
    data['Date/Time'] += datetime.timedelta(hours=8) # to GMT+8
    data['Date'] = pd.to_datetime(data['Date/Time']).dt.date
    data['Time'] = pd.to_datetime(data['Date/Time']).dt.time
    data['Date'] = pd.to_datetime(data['Date'])
    data['Shift'] = ""
    data['Failure Mode'] = data['Desc2']
    data['Failure Mode']= data['Failure Mode'].str.replace('\d+', '')
    data['TimeofAlarm']=data["Date/Time"].dt.hour
    data=data.loc[(data['Attribute'] == 'ALARMS.MACK') | (data["Event Type"]=="ALARM") & (data["State"]=="ACT/ACK") | (data["State"]=="ACT/UNACK")]
    data=data.drop(['FracSec', 'Ord', 'IsArchived','Event_SubType'],axis=1)
    data['Week_category']='This week'
    # Shift
    data.loc[(data['TimeofAlarm'] < 7) | (data['TimeofAlarm'] == 23), 'Shift'] ='N'
    data.loc[(data['TimeofAlarm'] >= 7) & (data['TimeofAlarm'] < 15) , 'Shift'] ='M'
    data.loc[(data['TimeofAlarm'] >= 15) & (data['TimeofAlarm'] < 23) , 'Shift'] ='A'
    print(f'Retrieved!')
    ### filter by alarm state too
    return data

In [3]:
latest_date='10-Jan-22.txt'

In [4]:
df_alm=get_csv(latest_date)

Retrieving 10-Jan-22.txt...
Retrieved!


# Get Team Number and Shift

In [5]:
def get_teams(df):
    
    df=df_alm
    
    df_MA=df.loc[df['Shift'] != 'N']
    
    df_MA=pd.merge(df_MA,calendar, on=['Date', 'Shift'], how='inner')
    
    #fixing night shift date to get correct Shift Team

# Time of alarm at 23

    df_night23=df.loc[(df['Shift'] == 'N') & (df['TimeofAlarm'] == 23)]

    df_night23=pd.merge(df_night23,calendar, on=['Date', 'Shift'], how='inner')

# time of alarm before 7 (need to minus 1 day)

    df_night7=df.loc[(df['Shift'] == 'N') & (df['TimeofAlarm'] < 7)]

    df_night7['Date']= df_night7['Date'] - datetime.timedelta(1)

    df_night7=pd.merge(df_night7,calendar, on=['Date', 'Shift'], how='inner')

    df_night7['Date']= df_night7['Date'] + datetime.timedelta(1)

    df_night=pd.concat([df_night23, df_night7])
    
    df=pd.concat([df_night,df_MA])
    
    df['Date']=df['Date'].astype(str)
    
    return df

In [6]:
df_alm=get_teams(df_alm)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
df_alm.head()

,Date/Time,Event Type,Category,Area,Node,Unit,Module,Module Description,Attribute,State,Level,Desc1,Desc2,Date,Time,Shift,Failure Mode,TimeofAlarm,Week_category,Team
0,2022-01-03 23:21:49,ALARM,NaN,PB1_TRAIN5,PB1CT020,T8521,ZAL-2289599,T8521 Chargepoint ZSL,DI_ALM,ACT/UNACK,11-WARNING,Digital Alarm,"Value OPEN, Normal CLOSED",2022-01-03,23:21:49,N,"Value OPEN, Normal CLOSED",23,This week,2
1,2022-01-03 23:23:48,ALARM,NaN,PB1_TRAIN5,PB1CT020,T8521,ZAL-2289599,T8521 Chargepoint ZSL,DI_ALM,ACT/UNACK,11-WARNING,Digital Alarm,"Value OPEN, Normal CLOSED",2022-01-03,23:23:48,N,"Value OPEN, Normal CLOSED",23,This week,2
2,2022-01-03 23:29:39,CHANGE,USER,PB1_TRAIN5,PB1WS013,T8521,ZAL-2289599,T8521 Chargepoint ZSL,ALARMS.MACK,NaN,NaN,WLK39042,NEW VALUE = 1,2022-01-03,23:29:39,N,NEW VALUE =,23,This week,2
3,2022-01-03 23:29:40,CHANGE,USER,PB1_TRAIN5,PB1WS013,T8521,ZAL-2289599,T8521 Chargepoint ZSL,ALARMS.MACK,NaN,NaN,WLK39042,NEW VALUE = 1,2022-01-03,23:29:40,N,NEW VALUE =,23,This week,2
4,2022-01-03 23:30:02,ALARM,DEVICE,PB1_LAM3_FAT4,PB1CT006,V79,QX-205596,V-79 Conductivity analyser,ADVISE_ALM,ACT/ACK,07-ADVISORY,ADVISE,Transducer Block Out Of Service,2022-01-03,23:30:02,N,Transducer Block Out Of Service,23,This week,2


# Filtered Data to CSV

In [ ]:
latest_date_filtered= f"{latest_date} - Filtered.txt"
source2="//Siwdsntv002/SG_PSC_SG1_PROJECT_NON-GMP/H0137_VS1 Alarms Data Mining_DL/PowerBI//CSV 2021/"

In [ ]:
df_alm.to_csv(source2+latest_date_filtered)

# Concat Historical Data and Latest data

In [ ]:
history=pd.read_csv()

In [ ]:
history.tail()

In [ ]:
history.info()

In [ ]:
# Replace This week to Last week in Historical Data
history['Week_category'] = history['Week_category'].replace(['This week'],'Last week')

In [ ]:
history.shape

# Check for Duplicates before Concat

In [ ]:
# Check Last Date
def concat(x,y):
    x_Date=x.iloc[-1:,14:15].values.tolist()
    y_Date=y.iloc[-1:,14:15].values.tolist()
# Check if last row dates of both df are the same
    if x_Date == y_Date:
         raise ValueError('Duplicate found, Please do not attempt to Duplicate')
    else: 
         x=pd.concat([x,y], sort=False)
    return x
        

In [ ]:
history=concat(history, df_alm)

In [ ]:
history['Week_category'].value_counts()

In [ ]:
history.to_csv("xxx", index=False)

# Extracting Information from Shift Report

In [8]:
#SHIFT REPORTER
from docx2python import docx2python
import re
import datetime
from IPython.display import Markdown
import pandas as pd
import numpy as np
 
#!pip install docx2python-1.27.1-py3-none-any.whl
#!pip install fuzzywuzzy
 
def flatten(x): 
    '''external function to recursively flatten lists'''
    result = []
    for el in x:
        if hasattr(el, "__iter__") and not isinstance(el, (str, bytes)):
            result.extend(flatten(el))
        else:
            if 'href' not in el:
                result.append(el.replace('\xa0',''))
    return result
 
def truncate_report(report,split_phrase1,split_phrase2):#=None):
    '''func used in __init__ method to truncate report before split_phrase'''
    try:
#         if split_phrase2 is not None:
#             print('in 1')
#             return report[report.index(split_phrase1)+1:] ## to remove
        return report[report.index(split_phrase1)+1:report.index(split_phrase2)]
    except:
        return report

def get_keywords(df_alarms, module):
    '''Takes an alarms dataframe and module as string
        Returns regex pattern of Module | Unit | Module Description as string
        To be used to search shiftreport
    '''
    # regex can be improved for edge cases with TIA-xxxx_CPM for certain modules
    
    keywords = [module.replace('-','')]

    unit_pattern = re.compile(r"[a-zA-z]{1,}-?\d{1,}")
    try:
        for col in ['Unit','Module Description']:
            results = df_alarms[df_alarms['Module'] == module][col].unique().tolist()

            # for module description, use re to extract potential units i.e. M1234, HE101
            if col == 'Module Description':
                for result in results:
                    keywords += re.findall(unit_pattern,result)

            keywords += results #concat all to list
        keywords = set([kw.replace('-','') for kw in keywords if type(kw)==str ]) # remove duplicates


        kw_pattern = fr"{'|'.join(keywords)}"
    except:
        #to return module for cases where description etc are NaN. This will prevent subsequent errors
        kw_pattern = keywords
        
    return kw_pattern
 

In [9]:
class shift_report:
    '''class shift report is initiated by (filename)
    methods:
        shift_report()
 
    '''
    def __init__(self,filename):
        
        # Regex pattern for date extraction
        pattern = re.compile(r"\d+-[a-zA-Z]+-\d+,? ([Mm]orning|[Aa]fternoon|[Nn]ight)\s*$")
        
        doc2x = docx2python(filename)
        self.shiftreport = flatten(doc2x.body) # recursively flatten list
        
        #docx created on
        #self.created_on = #datetime.datetime.strptime(
            #doc2x['created'],"%Y-%m-%dT%H:%M:%SZ").strftime("%d-%b-%Y %H:%M") 
     
        self.dates = list(filter(pattern.match, self.shiftreport))
        # for each date indexed by i, create list within list to be zipped to dataframe
        self.shiftreport = [self.shiftreport[self.shiftreport.index(self.dates[i])+1:self.shiftreport.index(
            self.dates[i+1])] if i < (len(self.dates)-1) else
            self.shiftreport[self.shiftreport.index(self.dates[i])+1:] for i in range(len(self.dates))
                       ] 
 
        srdf = pd.DataFrame(zip(self.dates,self.shiftreport),columns=['Date','Report'])
        srdf['Report'] = srdf['Report'].apply(lambda x: list(filter(None,x)))
 
        srdf['Reported By'] = srdf['Report'].apply(lambda x: ' '.join(x[1:4]))
 
        #Truncate report
        srdf['Report'] = srdf.Report.apply( # Truncate report here
            lambda x: truncate_report(x,"Plant Engineering Work Request  & Details ( WR Raised no. ) ",
                                      "Abnormalities of trends observed:")) #"Process Detail for all batches offloaded for trains"
        
        # Remove white spaces and '-'
        srdf['Report'] = srdf['Report'].apply(
            lambda x: [el.replace('-','') for el in x if el not in [' ','']])
                                                                # remove white spaces / ##### TO MAKE THIS REGEX #####
        
        
        self.df = srdf.copy()
#         print(f'File loaded.. \
#             Exported on {self.created_on}, dates from {self.dates[0]} to {self.dates[-1]}')
        self.melt_dataframe(self.df)
        print(f'Dataframe melted')
        
    def melt_dataframe(self, df):
        '''Further processing for self.df to melt into single rows'''
        
        df = df.merge(df['Report'].apply(pd.Series),
                    right_index=True,left_index=True).melt(id_vars=['Date','Reported By','Report'],
                                                           value_name="Text",var_name="Var")

        df = df.drop(['Report','Var'],axis=1)
        
        df['Shift'] = df['Date'].str[9:]
        df['Date'] = df['Date'].str[:9]
        df['Date'] = pd.to_datetime(df['Date'],errors='coerce')
        df['Shift_'] = df['Shift'].map({'Morning':'01-M',
                          'Afternoon':'02-A','Night':'03-N'})
        
        df = df.sort_values(['Date','Shift_']).drop('Shift_',axis=1)
        df = df.dropna(subset=['Date','Text'])
        df = df.drop_duplicates(subset=['Text'],keep='first')
        
        cols = ['Date','Shift','Reported By','Text']
        self.df = df.loc[:,cols]
    
    def search_dataframe(self,keywords):
        '''Takes in keywords as regex, returns self.df filtered by keyword matches
        '''
        print(f'Searching for ... {keywords}')
        mask = self.df['Text'].str.contains(keywords,case=False)
        results = self.df[mask].copy()
        return results

In [16]:
df_alm_shift = df_alm.loc[(df_alm['Level'].isin(['15-CRITICAL','11-WARNING']))]

N = 7
top_N_modules = df_alm_shift.Module.value_counts()[:N].index.to_list()
top_N_modules

['QIA-205596',
 'TIA-204005',
 'TIA-360149',
 'TIA-2284506',
 'CVO-200072',
 'M8914_DRUMCHG_EM',
 'XA-2284040']

In [17]:
#To be continued
# to develop for multiple shift-report objects for different months (sr)
#os.listdir()
path = ""
filename = "Dec 2021.docx"
#doc2x = docx2python(filename)

#2x loop for 2 months (overlapping)

sr = shift_report(path+filename)
results = []
for module in top_N_modules:
    result = sr.search_dataframe(get_keywords(df_alm,module)).copy()
    result['Module'] = module
    results.append(result)

results = pd.concat(results)

# to send to excel file, remove duplicates for different weeks?
results.to_clipboard(index=False)

Dataframe melted
Searching for ... V79|V79 Conductivity analyser|QIA205596
Searching for ... TIA204005|V400 HEAT TRACING TEMP|V400
Searching for ... BRINE SUPPLY TEMPERATURE|PB1_UTILS|TIA360149
Searching for ... HE8570 BRR TX|R8570|TIA2284506|HE8570
Searching for ... CVO200072|V1 Top Jkt Vent CVO|V1
Searching for ... M8914 Drum Charge EM|M8914_DRUMCHG_EM|M8914
Searching for ... XA2284040|R8460 Agi Seal|R8460


In [19]:
# to develop for multiple shift-report objects for different months (sr)
#os.listdir()
path = "
filename = "Jan 2022.docx"
#doc2x = docx2python(filename)

#2x loop for 2 months (overlapping)

sr = shift_report(path+filename)
results = []
for module in top_N_modules:
    result = sr.search_dataframe(get_keywords(df_alm,module)).copy()
    result['Module'] = module
    results.append(result)

results = pd.concat(results)

# to send to excel file, remove duplicates for different weeks?
results.to_clipboard(index=False)

Dataframe melted
Searching for ... V79|V79 Conductivity analyser|QIA205596
Searching for ... TIA204005|V400 HEAT TRACING TEMP|V400
Searching for ... BRINE SUPPLY TEMPERATURE|PB1_UTILS|TIA360149
Searching for ... HE8570 BRR TX|R8570|TIA2284506|HE8570
Searching for ... CVO200072|V1 Top Jkt Vent CVO|V1
Searching for ... M8914 Drum Charge EM|M8914_DRUMCHG_EM|M8914
Searching for ... XA2284040|R8460 Agi Seal|R8460


# Misc

In [ ]:
# concat large data

files = [file for file in os.listdir(source) if '21.txt' in file]
df_alm = pd.concat([(get_csv(file)) for file in files])
df_alm.sort_values('Date/Time',inplace=True)
df_alm.head()
